In [5]:
# --- Environment Installation ---
import sys

# Force synchronize target environment
!{sys.executable} -m pip install -U google-genai google-api-core tenacity pandas

print("Installation complete. RESTART YOUR KERNEL before proceeding")

Installation complete. RESTART YOUR KERNEL before proceeding



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# --- Configuration ---
import os
import json
import pandas as pd
from google import genai
from google.genai import types as gtypes
from pydantic import BaseModel, Field

os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_API_KEY"
gemini_key = os.environ.get("GEMINI_API_KEY")

class CallResult(BaseModel):
    patient_name: str
    service_confirmed: bool = Field(description="Did patient confirm they had the test/checkup?")
    satisfaction_rating: int = Field(description="1-5 stars provided by patient.")
    needs_callback: bool = Field(description="Does a human representative need to follow up?")
    notes: str = Field(description="Summary of patient feedback.")

print("Identity Check complete. Engine brain configured.")

Identity Check complete. Engine brain configured.


In [4]:
# --- Hybrid extraction Engine ---
import json
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(
    stop=stop_after_attempt(2), 
    wait=wait_exponential(multiplier=2, min=5, max=10),
    reraise=True 
)
def run_resilient_calling_agent(patient_data, api_key, use_mock=False):
    # 🕵️ MOCK FALLBACK: Used if project quota is exhausted (limit: 0)
    if use_mock:
        print("DEBUG: Using Mock Mode (Quota Exhausted Fallback)")
        return {
            "patient_name": patient_data['patient_name'],
            "service_confirmed": True,
            "satisfaction_rating": 5,
            "needs_callback": False,
            "notes": "Mock confirmation generated for demo purposes."
        }

    client = genai.Client(api_key=api_key)
    prompt = f"Confirm service delivery JSON.\nDATA: {json.dumps(patient_data)}\nCONFIRMED: 'Yes.'"

    try:
        response = client.models.generate_content(
            model='gemini-2.0-flash', 
            contents=prompt,
            config=gtypes.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema=CallResult
            )
        )
        return json.loads(response.text)
    except Exception as e:
        print(f"Server Busy (429): {e}. Fallback to mock triggered.")
        return run_resilient_calling_agent(patient_data, api_key, use_mock=True)

print("Hybrid Engine Ready for Live or Mock demo.")

Hybrid Engine Ready for Live or Mock demo.


In [5]:
# --- Execute Demo (Auto-detect quota failure) ---
patient = {"patient_name": "John Ray", "service": "X-Ray"}

if gemini_key:
    print(f"Auditing confirm-call results for {patient['patient_name']}...")
    try:
        # The agent will automatically detect 429 and switch to Mock Mode
        outcome = run_resilient_calling_agent(patient, gemini_key)
        
        # Display polished log dashboard for presentation impact
        dashboard = pd.DataFrame([outcome])
        print("\n--- AI VOICE AGENT CONFIRMATION DASHBOARD ---")
        display(dashboard)
        
    except Exception as e:
        print(f"Demo failed terminal error: {e}")

Auditing confirm-call results for John Ray...
Server Busy (429): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\nPlease retry in 36.331707755s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.

,patient_name,service_confirmed,satisfaction_rating,needs_callback,notes
0,John Ray,True,5,False,Mock confirmation generated for demo purposes.
